#ARS TLS practical part 4: structural metrics from tree point clouds with ITSMe

In the last part of this practical we want to calculate structural metrics such as DBH, tree height, crown area from the tree point clouds. We will do this using the R package ITSMe. 






Before we can start we need to mount our google drive again. 

‼️You have to run the next block of code in **Python**. Therefore you need to set the runtime type to python:

> Go to **Runtime > Change runtime type > Runtime type** → choose **Python** → click **Save**.

In [ ]:
# ==========================================================
# Mount Google Drive (run once per session)
# ==========================================================

from google.colab import drive
drive.mount('/content/drive')

‼️ The following part of the practical we will use **R** code. Therefore you need to set the runtime type to R:

> Go to **Runtime > Change runtime type > Runtime type** → choose **R** → click Save.

## Checking package Installation

To check if everything went well during the installation of ITSMe when you ran the preparation script, run the following block of code. This will start installing ITSMe if you did not install it before.


In [ ]:
# ==========================================================
# SETUP STEP 2 — Configure persistent R library
# ==========================================================

drive_lib <- "/content/drive/MyDrive/Rlib"

# Create library folder if it doesn't exist
if (!dir.exists(drive_lib)) {
  dir.create(drive_lib, recursive = TRUE)
}

# Tell R to use Drive library FIRST
.libPaths(drive_lib)

cat("Current library paths:\n")
print(.libPaths())

# ==========================================================
# SETUP STEP 3 — Install ITSMe if needed
# ==========================================================

if (!requireNamespace("ITSMe", quietly = TRUE)) {

  cat("\nITSMe not found. Installing (this may take some time the first time)...\n")

  devtools::install_github(
    "lmterryn/ITSMe",
    build_vignettes = FALSE,
  )

  cat("\nInstallation complete!\n")

} else {
  cat("\nITSMe already installed — loading package.\n")
}

library(ITSMe)

cat("\nITSMe successfully loaded!\n")


## ITSMe introduction
ITSMe is an R package developed to be able to quickly and easily extract individual tree structural metrics (like diameter at breast height, tree height, crown area, etc.) from **individual tree** point clouds.

You can find more info on the ITSMe package and the functions it provides [here](https://lmterryn.github.io/ITSMe/index.html). Under the reference tab you can find all the functions provided by the ITSMe R package. If you click on these function you can find more info about them too. During this practical we will be using the functions:


*   [read_tree_pc](https://lmterryn.github.io/ITSMe/reference/read_tree_pc.html)
*   [tree_height_pc](https://lmterryn.github.io/ITSMe/reference/tree_height.html)
*   [dbh_pc](https://lmterryn.github.io/ITSMe/reference/dbh_pc.html)
*   [diameter_slice_pc](https://lmterryn.github.io/ITSMe/reference/diameter_slice_pc.html)
*   [classify_crown_pc](https://lmterryn.github.io/ITSMe/reference/classify_crown_pc.html)
*   [projected_area_pc](https://lmterryn.github.io/ITSMe/reference/projected_area_pc.html)






## Importing point cloud data
Before we can extract metrics from a tree point cloud we need to import it into our environment. Depending on the size of the point cloud this can take a few seconds to a minute.

In [ ]:
tree1 <- read_tree_pc(path = "/content/drive/MyDrive/ARS_TLS_course/data/corrected_tree/corrected_tree_29.ply") #change the path to where you stored your point cloud in your drive.

If you print out the variable tree you can see that it is a dataframe with columns X, Y, Z which are the coordinates of the points (rows).

In [ ]:
tree1

## Calculating structural metrics from the tree point cloud

### Tree height
**tree_height_pc()** calculates the difference in the Z value between the highest and lowest point of the point cloud.

Calculate the tree height using `tree_height_pc(pc, plot)`
* `pc` = point cloud as a data.frame which is the output of read_tree_pc()
* `plot` = makes a figure when TRUE

In [ ]:
options(warn = -1)

#Calculate tree height
h1 <- tree_height_pc(pc = tree1, plot = TRUE)

#Print out the tree height
cat(sprintf("This tree is %.2f m tall\n", h1$h))

#Save the tree height image
ggplot2::ggsave(filename = "/content/drive/MyDrive/ARS_TLS_course/outputs/my_tree1_treeheight.png", plot = h1$plotXZ) #change the path



> 🤔 **Reflection 1:** Which scenarios (regarding data collection, processing, ...) can you think of that could cause errors in this tree height estimation? For example, when there occlusion of the stem due to a bush surrounding the stem. Think of 2 other scenarios and if/how this could be mitigated.

> 🤔 **Reflection 2:** How accurate and precise do you think tree height is from TLS point clouds? Why?




### Diameter at breast height
DBH can be calculated in different ways. **dbh_pc()** includes two options:

1. fitting a circle to a horizontal slice at breast height → **DBH** is the diameter of this circle.

2. fitting a concave hull to a horizontal slice at breast height → **fDBH** is the diameter of a circle with the same area as the concave hull.

Calculate the DBH and fDBH using `dbh_pc(pc, functional, plot)`  
* `pc` = point cloud as a data.frame which is the output of read_tree_pc()
* `functional` = also calculates the fDBH when TRUE.
* `plot` = makes a figure when TRUE




In [ ]:
#Calculate DBH
dbh1 <- dbh_pc(pc = tree1, functional = TRUE, plot = TRUE)

#Save the DBH image
ggplot2::ggsave(filename = "/content/drive/MyDrive/ARS_TLS_course/outputs/my_tree1_dbh.png", plot = dbh1$plot)

If the default parameters in `dbh_pc()` don’t give the desired result try playing around with some of the parameter values:
* `thresholdR2`
* `slice_thickness`
* `concavity`

Look up what these parameters do [here](https://lmterryn.github.io/ITSMe/reference/dbh_pc.html).

In [ ]:
#Calculate DBH
dbh1a <- dbh_pc(pc = tree1, thresholdR2 = 0.002, functional = TRUE, plot = TRUE) #thresholdR2 is mainly important when tree has branches at breast height

dbh1b <- dbh_pc(pc = tree1, slice_thickness = 0.2, functional = TRUE, plot = TRUE) #Increase or decrease the thickness of the slice taken, mainly important when there's low point density (take thicker slice) or when tree stem is slanted (take thinner slice)

dbh1c <- dbh_pc(pc = tree1, concavity = 3, functional = TRUE, plot = TRUE) #Increase concavity value to get a hull closer to the points.

> 🤔 **Reflection 3**: When would it be best to use the circle fitting method and when would be best to use the concave hull method?

> 🤔 **Reflection 4**: What could cause problems with the diameter measurements using each of these methods?

### Stem diameter

The stem diameter at any other height along the stem can be measured using the **diameter_slice_pc()**.

Calculate the DBH and fDBH at different heights using `diameter_slice_pc(pc, functional, plot)`:  
* `pc` = point cloud as a data.frame which is the output of read_tree_pc()
* `functional` = also calculates the fDBH when TRUE.
* `slice_height` = height at which the diameter will be calculated



In [ ]:
#Calculate diameter at different heights
d100 <- diameter_slice_pc(pc = tree1, slice_height = 1, functional = TRUE, plot = TRUE)

d150 <- diameter_slice_pc(pc = tree1, slice_height = 1.5, functional = TRUE, plot = TRUE)

d190 <- diameter_slice_pc(pc = tree1, slice_height = 1.9, functional = TRUE, plot = TRUE)

> 🤔 **Reflection 5**: Which previous processing steps can have a impact on these diameter measurement?

> 🤔 **Reflection 6**: What are the advantages of measuring diameters with TLS compared to traditional measurement?


### Crown classification
The tree point cloud can be split in two parts, crown and stem using **classify_crown_pc()** based on where the first branch emerges.

Split your tree point cloud into crown and stem using `classify_crown_pc(pc, thresholdbranch, plot)`
* `pc` = point cloud as a data.frame which is the output of read_tree_pc()
* `thresholdbranch` = default 1.5 (increase/decrease this value if the split happens too low/high)
* `plot` = makes a figure when TRUE

Put the crown points in a new object.



In [ ]:
#Split the tree in crown and stem points
tree1_classified <- classify_crown_pc(pc = tree1, thresholdbranch = 1.5, plot = TRUE) #Play around with the thresholdbranch value

#Save the crown points in a new object
crown1 <- tree1_classified$crownpoints

#Save the classification image
ggplot2::ggsave(filename = "/content/drive/MyDrive/ARS_TLS_course/outputs/my_tree1_classify.png", plot = tree1_classified$plot)

### Crown area
The crown area is calculated with `projected_area_pc()` as the area of the concave hull fitted to the XY-projection of the crown points.

Calculate the crown area using `projected_area_pc(pc, concavity, plot)`:
* `pc` = point cloud as a data.frame of the **crown points** output of running classify_crown_pc() on the tree point cloud
* `concavity` = default 2 (play around with this value)
* `plot` = makes a figure when TRUE





In [ ]:
#Calculate the crown area
ca1 <- projected_area_pc(pc = crown1, concavity = 3, plot = TRUE) #Play around with the concavity value

#Save the classification image
ggplot2::ggsave(filename = "/content/drive/MyDrive/ARS_TLS_course/outputs/my_tree1_crownarea.png", plot = ca1$plot)


> 🤔 **Reflection 7**: What can influence the crown area value that you calculate?

> 🤔 **Reflection 8**: Is it always necessary to split the tree point cloud into crown and stem point clouds? If not, in which cases is it necessary?


## ⏱️ Time left? 

You have calculated these metrics now based on 1 nicely manually corrected tree point cloud. To gain more insight to answer the reflection questions in this notebook you should run the code also on the uncorrected tree point cloud of that tree as well as several other (un)corrected trees. This will give you an idea of the impact of instance segmentation and other factors that influence the calculation of these metrics. 